Characterize traffic patterns from the simulation:
What are the traffic hot spots? Is there any congestion?
What are the travel times? (How) do they vary throughout the day?
What are busy times? How well do they match the commute pattern from NHTS?
How do speeds vary spatially and temporally?
What are the most popular roads?
Can you draw conclusions about the simulation setup from the output?


In [7]:
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import numpy as np
from zipfile import ZipFile
import glob
import zipfile
from sklearn.neighbors import BallTree
import numpy as np
from shapely.geometry import LineString

In [10]:
def read_gdf_from_zip(zip_fp):
    """
    Reads multiple csv files from ZipFile into a Pandas dataframe.
    """
    for zip_file in glob.glob(zip_fp):
        zf = zipfile.ZipFile(zip_file)
        dfs = [pd.read_csv(zf.open(f),sep=",") for f in zf.namelist()]
        dfs.encode('utf-8').strip()
        sim_df = pd.concat(dfs,ignore_index=True)
    
    return sim_df

In [15]:
snapshot = read_gdf_from_zip('data/vehicle_data/Simulation_Snapshot/Archive.zip')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 1: invalid start byte

In [ ]:
def start_end(sim_df):
    """
    Reads a spatial dataset from ZipFile into GeoPandas. Assumes that there is only a single file (such as GeoPackage)
    inside the ZipFile.
    """
    sim_snap_df = sim_df.groupby('VEHICLE')   
    sim_start = pd.DataFrame(sim_snap_df.head(1))
    sim_end = pd.DataFrame(sim_snap_df.tail(1))
    sim_end.X_COORD = pd.to_numeric(sim_end.X_COORD)
    sim_end.Y_COORD = pd.to_numeric(sim_end.Y_COORD)
    sim_start.X_COORD = pd.to_numeric(sim_start.X_COORD)
    sim_start.Y_COORD = pd.to_numeric(sim_start.Y_COORD)
    sim_start_gdf = gpd.GeoDataFrame(sim_start, geometry=gpd.points_from_xy(sim_start.X_COORD,sim_start.Y_COORD))
    sim_start_gdf.crs="EPSG:26916"
    sim_start_wgs84 = sim_start_gdf.to_crs(epsg=4326)
    sim_end_gdf = gpd.GeoDataFrame(sim_end, geometry=gpd.points_from_xy(sim_end.X_COORD,sim_end.Y_COORD))
    sim_end_gdf.crs="EPSG:26916"
    sim_end_wgs84 = sim_end_gdf.to_crs(epsg=4326)
    sim_start_wgs84 = sim_start_wgs84.reset_index(drop=True)
    
    return sim_start_wgs84, sim_end_wgs84

In [ ]:
sim_start,sim_end = start_end(snapshot)